# E11 멋진 작사가 만들기

## Step 0. 모듈

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import glob
import os
from sklearn.model_selection import train_test_split

## Step 1. 데이터 다운로드

* wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
* unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics

## Step 2. 데이터 읽기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:15])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion", "I can't get no relief Business men, they drink my wine", 'Plowman dig my earth', 'None were level on the mind', 'Nobody up at his word', 'Hey, hey No reason to get excited', 'The thief he kindly spoke', 'There are many here among us', 'Who feel that life is but a joke', "But, uh, but you and I, we've been through that", 'And this is not our fate', "So let us stop talkin' falsely now", "The hour's getting late, hey All along the watchtower"]


## Step 3. 데이터 정제  
* 문장을 토큰화했을 경우 토큰 개수 15개 넘을 시 잘라냄

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men , they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey , hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) #토큰 개수 15개

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  65 280 ...   0   0   0]
 [  2 107   6 ...   0   0   0]
 [  2  65  16 ...   0   0   0]
 ...
 [  2  39  39 ...   0   0   0]
 [  2   5  22 ...   0   0   0]
 [  2  39  39 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fcaa3de9410>


In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [6]:
src_input = tensor[:, :-1] # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다.
tgt_input = tensor[:, 1:] # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  65 280  27  99 528  19  85 778  93   3   0   0   0]
[ 65 280  27  99 528  19  85 778  93   3   0   0   0   0]


## Step 4. 평가 데이터셋 분리

In [7]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset
#데이터 전처리 끝

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기
* 모델의 Embedding Size, Hidden Size 조절하여 10 Epoch안에 val_loss 값을 2.2수준으로 줄일 수 있는 모델 설계

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 4.4241818e-04, -3.9798433e-05, -2.5222445e-04, ...,
          1.4962131e-04,  7.8246550e-05,  2.0173859e-05],
        [ 5.3515943e-04, -3.1960994e-05, -4.5926372e-05, ...,
          3.9831811e-04, -1.3332973e-04,  9.9574965e-05],
        [ 5.8413588e-04, -1.6642768e-04,  7.9467958e-05, ...,
          6.0163817e-04, -6.1994983e-04,  5.8025360e-04],
        ...,
        [ 5.0016359e-04,  5.3034333e-04, -1.5058708e-03, ...,
         -5.6587893e-04,  2.7353494e-04,  5.4724119e-04],
        [ 4.2798588e-04,  9.6645148e-04, -2.0921095e-03, ...,
         -7.6897931e-04,  4.1347416e-04,  2.5786928e-04],
        [ 3.4619489e-04,  1.3182849e-03, -2.6700122e-03, ...,
         -1.0534760e-03,  4.7058676e-04,  7.5982854e-05]],

       [[ 4.4241818e-04, -3.9798433e-05, -2.5222445e-04, ...,
          1.4962131e-04,  7.8246550e-05,  2.0173859e-05],
        [ 9.6222089e-04,  1.1019529e-06, -3.5704474e-04, ...,
          1.5334945e-04, 

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  6295552   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 33,133,793
Trainable params: 33,133,793
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 89s 163ms/step - loss: 3.6467
Epoch 2/10
549/549 [==============================] - 91s 166ms/step - loss: 3.1404
Epoch 3/10
549/549 [==============================] - 92s 168ms/step - loss: 2.9527
Epoch 4/10
549/549 [==============================] - 91s 166ms/step - loss: 2.8074
Epoch 5/10
549/549 [==============================] - 94s 172ms/step - loss: 2.6784
Epoch 6/10
549/549 [==============================] - 92s 168ms/step - loss: 2.5583
Epoch 7/10
549/549 [==============================] - 90s 163ms/step - loss: 2.4451
Epoch 8/10
549/549 [==============================] - 90s 163ms/step - loss: 2.3387
Epoch 9/10
549/549 [==============================] - 83s 152ms/step - loss: 2.2361
Epoch 10/10
549/549 [==============================] - 78s 142ms/step - loss: 2.1375


In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '

아무래도 maxlen을 어떻게 처리할지가 관건이었다.

tokenize함수 tensor에 maxlen=15를 넣으면 15가 넘어갈 경우 그 문장의 뒤를 싹둑 잘라서 문장의 완성형 형태가 아니게 되므로 학습에 혼란을 주는 경우가 발생한다는 정보를 얻었다. 우선 Step 3에서 제안된 방식이 잘라내는 것이므로 여기서는 다른 선택을 하지 않았지만 다른 조원이 한 것처럼 15를 넘어가는 문장을 빼버리는 과정을 거치는 것도 나쁘지 않다고 생각했다.
하지만 그렇게 하기 위해서는 그 문장들의 개수가 몇개인지 확인하는 과정을 거치고 그 중요도를 파악한 후에 해야 한다는 생각이 들었고, 안그래도 진행이 늦은 입장에서 그것을 빨리 처리할 자신이 없었다.